# Binary Classification Twitter Sentiment Analysis in Python Using Logistic Regression

In [80]:
# Libraries for Necesary Data Structures
import numpy as np
import pandas as pd
import string
# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
# Natural Language Processing libraries
from wordcloud import WordCloud
from nltk import TweetTokenizer, PorterStemmer
from nltk.corpus import stopwords
# Machine Learning Tools
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

# Quick settings before starting
%matplotlib inline
sns.set_theme()

### Read in the training dataset as a Pandas DataFrame

In [81]:
df_train = pd.read_csv("data/train_dataset_20k.csv")
df_train.head()

,Sentiment,id,Date,Query,User,Tweet
0,4,2014351367,Wed Jun 03 01:14:33 PDT 2009,NO_QUERY,cathyleehart,@just_tam21 hahah its kind of the same as adel...
1,4,1984531826,Sun May 31 15:17:28 PDT 2009,NO_QUERY,carlawaslike,@Austinjamest sweeeet can't wait to go there w...
2,4,2185079853,Mon Jun 15 16:51:18 PDT 2009,NO_QUERY,Rommellll,You said you changed how wrong i proved you n...
3,4,1998128979,Mon Jun 01 18:02:23 PDT 2009,NO_QUERY,pauljopling,I can't wait for New Moon after seeing the tra...
4,4,2182994010,Mon Jun 15 13:54:26 PDT 2009,NO_QUERY,thisgoodlife,@nicolassa new drum carder? SWEET!


### By default, sentiments are labeled as "4" for positive and "0" for negative. For legibility purposes, I'm adjusting the training dataset by replacing the 4's with 1's, and also by getting rid of unnecessary columns.

In [82]:
df_train = df_train[["Sentiment", "id", "Tweet"]]
df_train.replace(4, 1, inplace=True)
df_train.head()

,Sentiment,id,Tweet
0,1,2014351367,@just_tam21 hahah its kind of the same as adel...
1,1,1984531826,@Austinjamest sweeeet can't wait to go there w...
2,1,2185079853,You said you changed how wrong i proved you n...
3,1,1998128979,I can't wait for New Moon after seeing the tra...
4,1,2182994010,@nicolassa new drum carder? SWEET!


In [83]:
df_train.info()
print(f"\nThere are {df_train.shape[0]:,} rows and {df_train.shape[1]:,} columns in our Tweet Training Set.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentiment  20000 non-null  int64 
 1   id         20000 non-null  int64 
 2   Tweet      20000 non-null  object
dtypes: int64(2), object(1)
memory usage: 468.9+ KB

There are 20,000 rows and 3 columns in our Tweet Training Set.


### To preprocess the tweets in our training dataset, we must do the following:
 1. Remove hyperlinks, punctuation, and hashtags
 2. Transform each Tweet into a list of its individual words
 3. Convert each word to Lower Case
 4. Delete stop words
 5. Stemming to equate words like "eat", "eats", and "eating" all to "eat"

In [84]:
def preprocess_tweets(df):
    """
    The preprocess_tweets() function goes through the following steps:
        1. Remove hyperlinks, punctuation, and hashtags
        2. Transform each Tweet into a list of its individual words
        3. Convert each word to Lower Case
        4. Delete stop words
        5. Stemming to equate words like "eat", "eats", and "eating" all to "eat"

    params:
        df : Pandas Dataframe containing our data

    @RETURN a 2D Python list with columns representing individual words and rows representing
        each individual Tweet
    """
    
    # Replace certain characters in each Tweet with empty spaces instead
    df["Tweet"] = df["Tweet"].str.replace(r"\$\w*", "", regex=True)
    df["Tweet"] = df["Tweet"].str.replace(r"https?:\/\/.*[\r\n]:", "", regex=True)
    df["Tweet"] = df["Tweet"].str.replace(r"http?:\/\/.*[\r\n]:", "", regex=True)
    df["Tweet"] = df["Tweet"].str.replace(r"#", "", regex=True)

    # Initialize a Tweet Tokenizer method that will convert Tweets to lower-case and eliminate Twitter handles
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

    # Make necessary declarations for upcoming 2D for-loop
    n = df.shape[0]
    tokens = []
    stopWords = stopwords.words("english")
    stemmer = PorterStemmer()
    stemmedTweets = []

    # Tokenize our tweets and insert them into the tokens numpy array
    for i in range(n):
        buffer = []
        tokens.append(tokenizer.tokenize(df_train.loc[i, "Tweet"]))

        # Remove stop words and words with punctuation 
        for tweet_word in tokens[i]:
            if tweet_word not in stopWords and tweet_word not in string.punctuation:
                #buffer.append(tweet_word)
                stem_word = stemmer.stem(str(tweet_word))
                buffer.append(stem_word)
            
        # Append the 2D List that we'll be returning with this function
        stemmedTweets.append(buffer)
        

    return stemmedTweets


### Apply the preprocess_tweets() function to return a 2D List containing cleaned tweets

In [85]:
trainingTweets = preprocess_tweets(df_train)

### Create a y-vector that includes all of the true positive and negative classifications for our training data

In [86]:
y_train = []
for i in range(df_train.shape[0]):
    y_train.append(1 if df_train["Sentiment"][i] == 1 else 0)

### Create a dictionary including all of our words and their positive and negative appearance frequencies in the training data

In [87]:
# Append the dictionary Data Structure every time the for loop comes across a new word, and update
# it by either incrementing the positive index [0] or negative index [1], depending on what kind
# of tweet it pops up in
dictionary = {}
for y, tweet in zip(y_train, trainingTweets):
    for token in tweet:
        if token not in dictionary:
            dictionary[token] = [1,0] if y==0 else [0,1]
        elif token in dictionary:
            if y==0:
                dictionary[token][0] += 1
            elif y==1:
                dictionary[token][1] += 1

### Create Positive appearances and Negative appearance lists for training tweets

In [93]:
posTrainingTweets = [[trainingTweets[i] for i in range(len(trainingTweets)) if y_train[i]==1 ]]
negTrainingTweets = [[trainingTweets[i] for i in range(len(trainingTweets)) if y_train[i]==0 ]]

### Feature Extraction: Create a (tweetNum) x 3 matrix to represent the following:
#### col[0] : all 1's to take into account a constant in a logistic regression model
#### col[1] : the number of training-set positive appearances for each word in each tweet
#### col[2] : the number of training-set negative appearances for each word in each tweet

In [88]:
def numerical_data(tweets, y, dictionary=dictionary):
    """
    The numerical_data() function translates tweets to measurable, quantitative data in a 2D matrix
    representing the following:
        column[0] : all 1's to denote where the logistic regression model stands in an empty
                    tweet
        column[1] : the sum of all positive appearances in the entire training set of each word that's
                    located in the tweet being evaluated
        column[2] : the sum of all negative appearances in the entire training set of each word that's
                    located in the tweet being evaluated
    
    params:
        tweets : a 2D-list where the outer elements represent individual tweets and the inner elements 
                 represent individual words within the tweet
        y : a list denoting the true sentiment score (1 or 0) of each tweet
        dictionary : a dict-type data structure representing all the words that appear in the training set,
                     as well as their total positive and negative appearances within the training set

    @RETURN a mx3 matrix where m represents the total number of tweets
    """
    m = len(tweets)
    X = np.zeros((m, 3))

    for i in range(m):
        X[i][0] = 1
        for word in tweets[i]:
            if word in dictionary:
                X[i][1] += dictionary[word][0]
                X[i][2] += dictionary[word][1]

    return X

    # m = len(tweets)
    # X = np.zeros((m, 3))

    # for i in range(m):
    #     X[i][0] = 1
    #     for word in tweets[i]:
    #         if word in dictionary:
    #             if dictionary[word][0] > dictionary[word][1]:
    #                 X[i][1] += 1
    #             elif dictionary[word][0] < dictionary[word][1]:
    #                 X[i][2] += 1

    # return X
        

In [89]:
X_train = numerical_data(trainingTweets, y_train)

### Use scikit-learn to generate logistic regression parameters, and test our model on the training set

In [90]:
# Create a LogisticRegression() object
logReg = LogisticRegression()

# Fit logistic regression parameters
logReg.fit(X_train, y_train)

# Calculate training accuracy and output it
trainingAccuracy = logReg.score(X_train, y_train)
print(f"Our training accuracy is {trainingAccuracy*100:.2f}%")

Our training accuracy is 69.25%


### Implement a Gradient Descent function to 